<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_test_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install llamaapi

In [ ]:
import json
from llamaapi import LlamaAPI
from google.colab import userdata

# Initialize the llamaapi with your api_token
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))

# Define your API request
api_request_json = {
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },
            "required": ["location", "days"],
        }
    ],
    "stream": False,
    "function_call": "get_current_weather",
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))


Exception: POST 401 Insufficient balance.

In [ ]:
!pip install --upgrade llamaapi

In [ ]:
import llamaapi

In [ ]:
#check llama api balance
balance = llama.get_balance()
print(f"Your current balance is: {balance}")

AttributeError: 'LlamaAPI' object has no attribute 'get_balance'